In [1]:
from google.colab import files

uploaded_model1 = files.upload()





Saving model3.csv to model3.csv


In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score, recall_score, f1_score, average_precision_score
from sklearn.decomposition import NMF
import tensorflow as tf
from tensorflow.keras.layers import Input, Embedding, Flatten, Dense, Concatenate
from tensorflow.keras.models import Model

# Load the dataset
data = pd.read_csv('model3.csv')

# Preprocess the data
# Convert labels to one-hot encoding
labels = data.iloc[:, 1:]
text_data = data['text']

# Train-test split
train_data, test_data = train_test_split(data, test_size=0.2, random_state=42)

# Matrix Factorization (NMF)
num_topics = 20
nmf = NMF(n_components=num_topics)
W = nmf.fit_transform(labels)
H = nmf.components_

# Neural Collaborative Filtering (NCF)
num_users = labels.shape[0]
num_items = labels.shape[1]
embedding_dim = 64

# Define NCF model
user_input = Input(shape=(1,))
item_input = Input(shape=(1,))

user_embedding = Embedding(num_users, embedding_dim)(user_input)
item_embedding = Embedding(num_items, embedding_dim)(item_input)

user_flat = Flatten()(user_embedding)
item_flat = Flatten()(item_embedding)

concat = Concatenate()([user_flat, item_flat])

dense_1 = Dense(64, activation='relu')(concat)
dense_2 = Dense(32, activation='relu')(dense_1)

output = Dense(1, activation='sigmoid')(dense_2)

ncf_model = Model(inputs=[user_input, item_input], outputs=output)
ncf_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train NCF model
user_indices = train_data.index
item_indices = range(num_items)
labels_train = labels.values

# Define positive and negative samples for training
def generate_samples():
    for user in user_indices:
        for item in item_indices:
            yield user, item, labels_train[user, item]

samples = np.array(list(generate_samples())).T
users_train = samples[0]
items_train = samples[1]
labels_train = samples[2]

ncf_model.fit([users_train, items_train], labels_train, epochs=100, batch_size=64)
# Save the trained model
ncf_model.save("ncf_model.h5")

# Download the saved model file
files.download("ncf_model.h5")

# Recommendation for a given text
def recommend_labels(text, n=5):
    # Predict labels using NCF model
    user_idx = np.array([0] * num_items)  # Assuming user index 0 for recommendation
    item_idx = np.array(range(num_items))
    scores = ncf_model.predict([user_idx, item_idx])

    # Sort labels by predicted scores and recommend the top n
    recommended_labels = np.argsort(-scores, axis=None)[:n]
    return recommended_labels

# Example: Recommend labels for a given text
text_user_input = "I helped my friend with a difficult situation. I am trying my best to save him overcome financial issues"
recommended_labels = recommend_labels(text_user_input, n=5)

# Print recommended labels
print("Recommended Microcredentials for the input text:")
for label in recommended_labels:
    print(f"Label {label}: {labels.columns[label]}")


Epoch 1/100
6596/6596 [==============================] - 40s 6ms/step - loss: 0.1174 - accuracy: 0.9655
Epoch 2/100
6596/6596 [==============================] - 39s 6ms/step - loss: 0.1007 - accuracy: 0.9692
Epoch 3/100
6596/6596 [==============================] - 39s 6ms/step - loss: 0.0881 - accuracy: 0.9724
Epoch 4/100
6596/6596 [==============================] - 38s 6ms/step - loss: 0.0795 - accuracy: 0.9748
Epoch 5/100
6596/6596 [==============================] - 36s 6ms/step - loss: 0.0708 - accuracy: 0.9775
Epoch 6/100
6596/6596 [==============================] - 37s 6ms/step - loss: 0.0627 - accuracy: 0.9802
Epoch 7/100
6596/6596 [==============================] - 37s 6ms/step - loss: 0.0555 - accuracy: 0.9822
Epoch 8/100
6596/6596 [==============================] - 37s 6ms/step - loss: 0.0495 - accuracy: 0.9841
Epoch 9/100
6596/6596 [==============================] - 36s 5ms/step - loss: 0.0440 - accuracy: 0.9857
Epoch 10/100
6596/6596 [==============================] - 36s 5m

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

5/5 [==============================] - 0s 3ms/step
Recommended Microcredentials for the input text:
Label 0: 111
Label 130: 651
Label 131: 652
Label 28: 216
Label 18: 142


In [3]:


# Define precision at K function
def precision_at_k(recommended_labels, true_labels, k):
    recommended_labels_at_k = recommended_labels[:k]
    relevant_labels = set(true_labels)
    num_relevant = len(relevant_labels)
    if num_relevant == 0:
        return 0
    num_relevant_at_k = len(set(recommended_labels_at_k).intersection(relevant_labels))
    precision = num_relevant_at_k / k
    return precision

# Define recall at K function
def recall_at_k(recommended_labels, true_labels, k):
    recommended_labels_at_k = recommended_labels[:k]
    relevant_labels = set(true_labels)
    num_relevant = len(relevant_labels)
    if num_relevant == 0:
        return 0
    num_relevant_at_k = len(set(recommended_labels_at_k).intersection(relevant_labels))
    recall = num_relevant_at_k / num_relevant
    return recall

# Define F1-score at K function
def f1_score_at_k(recommended_labels, true_labels, k):
    precision = precision_at_k(recommended_labels, true_labels, k)
    recall = recall_at_k(recommended_labels, true_labels, k)
    if precision + recall == 0:
        return 0
    f1_score = 2 * (precision * recall) / (precision + recall)
    return f1_score

# Define Mean Average Precision (MAP) function
def mean_average_precision(recommended_labels, true_labels):
    average_precision = 0
    num_relevant = 0
    for k, label in enumerate(recommended_labels):
        if label in true_labels:
            num_relevant += 1
            average_precision += num_relevant / (k + 1)
    if num_relevant == 0:
        return 0
    return average_precision / num_relevant

def evaluate_recommendations():
    # Initialize lists to store evaluation results
    precision_scores = []
    recall_scores = []
    f1_scores = []
    map_scores = []

    for index, row in test_data.iterrows():
        text = row['text']
        true_labels = row[1:].values  # Get true labels from the row

        # Generate recommendations
        recommended_labels = recommend_labels(text, n=5)  # You can adjust 'n' as needed

        # Calculate precision, recall, and F1-score at K
        precision = precision_at_k(recommended_labels, true_labels, 5)
        recall = recall_at_k(recommended_labels, true_labels, 5)
        f1 = f1_score_at_k(recommended_labels, true_labels, 5)

        # Calculate Mean Average Precision (MAP)
        map_score = mean_average_precision(recommended_labels, true_labels)

        # Append results to the lists
        precision_scores.append(precision)
        recall_scores.append(recall)
        f1_scores.append(f1)
        map_scores.append(map_score)

    # Calculate and print the average scores
    avg_precision = sum(precision_scores) / len(precision_scores)
    avg_recall = sum(recall_scores) / len(recall_scores)
    avg_f1 = sum(f1_scores) / len(f1_scores)
    avg_map = sum(map_scores) / len(map_scores)

    print(f"Average Precision at K: {avg_precision}")
    print(f"Average Recall at K: {avg_recall}")
    print(f"Average F1-Score at K: {avg_f1}")
    print(f"Average Mean Average Precision (MAP): {avg_map}")

evaluate_recommendations()


5/5 [==============================] - 0s 3ms/step
Average Precision at K: 0.2000000000000008
Average Recall at K: 0.5
Average F1-Score at K: 0.2857142857142845
Average Mean Average Precision (MAP): 1.0
